# Initialize

In [1]:
# import packages 

%load_ext autoreload
# OS Imports
import os
from os import listdir
from os.path import isfile, join
import shutil

# Garbage collection 
import gc
import sys

# Basic math/data processing packages
import numpy as np
from scipy import signal
import pandas as pd 
import matplotlib.pyplot as plt
import math
import matplotlib
import random
import time 
import datetime
from collections import deque

# Imports for CSV Processing
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm, PowerNorm
import matplotlib.style as mplstyle
import matplotlib.patches as mpatches
import pickle
import openpyxl

# Imports for Deep Learning for data preparation and SVM 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (mean_squared_error, confusion_matrix, 
    ConfusionMatrixDisplay, classification_report, 
    cohen_kappa_score, matthews_corrcoef)
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.decomposition import PCA

# Keras/TF Packages
import keras

from keras.regularizers import l1
from keras.models import Sequential, load_model
from keras import initializers
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import (Dense, Dropout, Activation, Flatten, Input, 
                          TimeDistributed, Reshape, Permute, Flatten, 
                          RepeatVector, Bidirectional, InputLayer,  
                          AlphaDropout, Normalization, MaxPooling2D, Embedding, 
                          ConvLSTM1D, Attention, TimeDistributed, LocallyConnected1D,
                          LSTM, GRU)
from keras.models import Model, load_model
from keras import optimizers
from keras.utils import plot_model
from keras.utils import to_categorical

from tensorflow.keras.optimizers import Nadam, Adam, SGD, Adadelta, Adamax, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import tensorflow as tf

print(keras.__version__)
print(tf.__version__)

print(matplotlib.__version__)

%load_ext tensorboard

2024-04-25 11:38:02.970537: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 11:38:07.348478: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-25 11:38:16.742048: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/bmoxon/anaconda3/envs/SWISC_310_tf210/lib/libfabric:
2024-04-25 11:38:16.743127: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dler

2.10.0
2.10.0
3.8.4


In [2]:
# Print GPU details if recognized
# Ensure GPU RAM is >10 GB
!nvidia-smi
print('GPU name: ', tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Setup CUDA
os.environ['CUDA_VISIBLE_DEVICES'] = '1'


Thu Apr 25 11:38:23 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.239.06   Driver Version: 470.239.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:81:00.0  On |                  N/A |
| 30%   27C    P8     1W /  38W |    244MiB /  1994MiB |     14%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

2024-04-25 11:38:24.379330: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/bmoxon/anaconda3/envs/SWISC_310_tf210/lib/libfabric:
2024-04-25 11:38:24.379445: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
# Consistent random seed selection improves reliability of Keras training performance
tf.random.set_seed(42)
np.random.seed(42)

In [64]:
def setPaths(data_path,epoch_length):
    # Set paths to preprocessed data based on epoch length
    match epoch_length:
        case 20:
            data_path='D:/Final_Export/'
            %cd $data_path

        case 4:
            data_path='D:/npy_no_z_4/'
            %cd $data_path

    # Set paths to Fourier-extracted data
    path_Fourier_scored=data_path+'npy_newest_scored/Feats_Fourier_and_PSD/'
    path_Fourier_unscored=data_path+'npy_newest_unscored/Feats_Fourier_and_PSD/'

    # Set output paths for data, variables, and results
    path_output=data_path+'T_CSV_Outputs/'
    path_variables=data_path+'T_Variables/'
    path_results=data_path+'T_Results_GPU/'

    # Create folders if they do not exist
    for path_iterable in [path_output,path_variables,path_results]:
        if os.path.exists(path_iterable)==False:
            os.mkdir(path_iterable)

    # Return paths
    return data_path, path_Fourier_scored, path_Fourier_unscored, path_output, path_variables, path_results


In [65]:
# Set paths based on base path where data is stored
data_path, path_Fourier_scored, path_Fourier_unscored, path_output, path_variables, path_results = setPaths(data_path='D:/', epoch_length=20)
%cd $data_path

D:\Final_Export
D:\Final_Export


In [72]:
# Number of hours of recording per file
train_recording_hours=12

# Length of epochs in seconds
train_recording_epoch_seconds=20

# Total number of epochs per recording
train_recording_epoch_count=int( (train_recording_hours*3600)/train_recording_epoch_seconds )

# List of all viable integer codes for scores. Any other integers in "score" array will be rejected
viable_scores=[1,2,3,4,5]

In [67]:
# Path to existing trained model file
modelpath=f'{path_results}Final Model/Models/BiLSTM_size_200___BiLSTM_200_win3.h5'

# Establish CSV folder structure
csv_folder=f'{data_path}CSV_Outputs/'

path_csv=f'{csv_folder}csv_classifier_prev_un_scored/'
path_holes=f'{csv_folder}csv_trainset_classifier_scored/'
mouse_sort_csvs=f'{csv_folder}csv_full/'
path_expert=f'{csv_folder}csv_trainset_expert_scored/'
path_figs=f'{csv_folder}figures/'
path_conf=f'{csv_folder}score_conf/'
path_excel=f'{csv_folder}excel/'

# Create CSV and output folders
for folder in [path_csv, path_holes, mouse_sort_csvs, path_expert, path_figs, path_excel, path_conf]:
  if os.path.exists(folder)==0:
    os.mkdir(folder)

print(mouse_sort_csvs)

D:/Final_Export/CSV_Outputs/csv_full/


# Define Functions

## Array Manipulation Functions

In [41]:
def consecutive(data, stepsize=1):
    # Simple function for finding blocks of consecutive numbers in an array
    # Find indicies where numbers differ
    inds =  np.where(np.diff(data) != stepsize)[0]+1
    # Split data on these indicies
    consec = np.split(data, inds)
    return consec, inds

In [42]:
def unique(list1): 
  
    # Intilize a null list 
    global unique_list  
    unique_list = [] 
    
    # Traverse for all elements 
    for x in list1: 
        # Check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
            
    return sorted(unique_list)

In [43]:
def list_prune_via_substrings(list_to_be_scanned, substring_list):
  result_list=[]

  if type(list_to_be_scanned[0])!=str:
    list_to_be_scanned=str(list_to_be_scanned)

  if type(substring_list[0])!=str:
    substring_list=[str(sub) for sub in substring_list]

  for scan_member in list_to_be_scanned:
    present=1
    for member in substring_list:
      if member in scan_member:
        present=0
    
    if present==1:
      result_list.append(scan_member)

  return result_list

In [44]:
def get_start_end(data_path):
    # Load .xlsx containing start and end dates of recording for each animal
    Excel=f'{data_path}/Start and End Dates.xlsx'
    df = pd.read_excel(Excel)
    start_dates=[]
    end_dates=[]
    # Transfer start and end dates to array for parsing
    for i in range(len(df)):
        start_dates.append(df['Name'][i])
        start_dates.append(df['Start'][i])
        end_dates.append(df['Name'][i])
        end_dates.append(df['End'][i])
    return start_dates,end_dates

def get_manual_dates(data_path):
    # Load .xlsx containing dates where animal data was manually scored
    Excel=f'{data_path}/Manual Dates.xlsx'
    df = pd.read_excel(Excel)

    return df

## Preprocessing Functions

In [45]:
def load_Fourier_xy(path, file_list, train_recording_epoch_count, viable_scores, standardize=True):

    total_files = len(file_list)  # Length of file list
    print(total_files)
    
    epochs = train_recording_epoch_count   
    total_epoch_count = int(epochs*(total_files)) # Total length of data array in 20 second epochs
    
    # Create displays at top of notebook output for reverse counter and current status
    dh1 = display(f'Items left: {total_files}',display_id=True)
    dh2 = display('Loading...',display_id=True)

    # Counting variable for progress
    file_counter = 0

    # Initialize arrays for x and y, total epochs in datset by number of features in vector 
    x = np.zeros((total_epoch_count, 100))
    y = np.zeros((total_epoch_count, 1))
    exclusion = False
    
    # Iterate over all Fourier transformed files
    for item in (sorted(file_list)):
        # Increment count down to completion of file loading
        total_files -= 1
        # Update display for files remaining
        dh1.update(f'Items left: {total_files}')

        # Load in epoch scores to check for invalid scores
        item = item.replace("x_ffnorm", "y_ffnorm") 
        current_file_y = np.load(f"{path}{item}")
        item = item.replace("y_ffnorm", "x_ffnorm") 

        # If invalid score is present, skip file
        for i in current_file_y:
            
            if i not in viable_scores:
                exclusion=True

        if exclusion==False:
            # Load Fourier feature vectors
            current_file_x = np.load(f"{path}{item}")

            # Load feature vectors for each file
            # If standardization is on, standardize values to 0-1 range for quicker training and evaluation
            if standardize==True:
                x[file_counter*epochs:((file_counter+1)*epochs)] = StandardScaler().fit_transform(current_file_x)
            else:
                x[file_counter*epochs:((file_counter+1)*epochs)] = current_file_x

            
            # load Sleep/Seizure score vectors
            y[file_counter*epochs:((file_counter+1)*epochs)] = current_file_y-1

            file_counter += 1
            # Update display for current status
            dh2.update(f'loaded x and y for {item}')
            # Periodically print progress 
            if file_counter%50 == 0:
                print(file_counter)

        else: 
            exclusion = False

    return x,y

In [47]:
def generate_sequences(input_array, windows, x_or_y, max_feats=None, make_seq=True):
    if make_seq==True:
        if x_or_y not in ['X', 'Y']:
            print("Please designate whether input is X or Y array using x_or_y parameter")
            return 

        for window in [windows]: # for loop to test varying window lengths
            classes=5

            shift=window*2 # All x variable rows will be sampled with sliding sequences 
            # If analyzing epoch 3, epochs from -window_length (0) to 
            # +window_length (6) around each epoch of interest will be sampled
            # Therefore, the windowing cannot begin earlier than epoch # [window_length]


            if x_or_y in ['X']:
                if max_feats == None:
                    # Get maximum numnber of features from X array if not specified
                    max_feats=input_array.shape[1]

                # Generate sliding window array 
                output_array=np.zeros((len(input_array)-shift,window*2+1,max_feats))
                
                # Cannot window on both sides unless first array taken has full window length prior to and after it, so for loop starts at window index and ends at end-window
                for i in range(window,len(input_array)-window):
                    output_array[i-window]=input_array[i-window:i+window+1,0:max_feats]
        
        
            if x_or_y in ['Y']:
                output_array=np.zeros((len(input_array)-shift, classes))
                for i in range(window,len(input_array)-window):
                    output_array[i-window]=input_array[i]


        gc.collect()
        return output_array

    elif make_seq==False:
        print('no sequence generated')
        return input_array

## Model Training Functions

In [48]:
def common_training_parameters():
    # Keras metrics for training evaluation
    metrics = [
        'accuracy',
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'),
        keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc', curve="PR"),
        keras.metrics.CategoricalCrossentropy(name='categorical_crossentropy')]

    # Batch size of training/evaluation = one 12 hour recording file worth of 20 second epochs
    batch_size = 2160

    # Set training optimizer
    optimizer=Nadam(learning_rate=0.00001)
    # Set regularization function for training
    activity_regularizer=l1(0.0001)

    return batch_size, metrics, optimizer, activity_regularizer

In [49]:
def norm_sklearn_classweight(y_train, mu=False):
    
    # Get maximum range of class labels 
    classes = unique(np.argmax(y_train,1))

    # Convert scores back from one-hot to ints
    y_train_classes = [np.argmax(z) for z in y_train]
    unique(y_train_classes)

    
    # Assign balanced class weights
    weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train_classes)
    print(weights)
    class_weight = {}
    ind = 0

    
    # Adjust weights based upon mu parameter (logarithmic normalization of class weights) or the minimum weight present
    mu=weights[0]/math.exp(1)
    for i in classes:
        j = int(np.where(classes==i)[0])
        score = math.log(weights[j]/mu)
        class_weight[j] = score 

    return class_weight

In [53]:
def create_train_Dense_model(X_train, X_test, y_train, y_test, class_weight, epochs, 
                       steps_per_epoch=None, layer_size=8, batch_size=2160, optimizer=Nadam(learning_rate=0.00001), metrics=['accuracy'], activity_regularizer=l1(0.0001), prefix="", layers=3, data_path=''):
        
        # Get dimensions for model input layer from input array
        data_width=X_train.shape[1]
   
        if len(X_train.shape)>2:
            data_depth=X_train.shape[2]
            shape=(data_width,data_depth)
        else: 
            data_width=X_train.shape[1]
            shape=(data_width,)

        # Create Sequential Model 
        model = Sequential()
        model.add(InputLayer(input_shape=shape))
        model.add(Dense(layer_size))
        model.add(Flatten())           
        model.add(Dense(5))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=metrics)
    
        model.build()
        model.summary()

        # Name model with logging based on training time and date and metrics to track
        model_name=f"Dense_size_{layer_size}_{prefix}_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        filepath = f"{model_name}"+"_weights-auc-{auc}-{categorical_accuracy:.6f}--val-{val_auc}-{val_categorical_accuracy:.6f}.h5"

        # Keras model checkpoint function
        mcp = ModelCheckpoint(f"./Checkpoints/{filepath}", 
                              monitor='val_loss', verbose=1, 
                              save_best_only=True, save_weights_only=False, mode='min')

        # Early stopping parameters
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                    patience=5, min_delta=0.001,restore_best_weights=True)

        # Tensorboard logging variables
        log_dir = f"./Logs/{model_name}"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        logpath=f"./Logs/"
        csv_path=f"{model_name}_model.csv"
        csv_logger=CSVLogger(logpath+csv_path)

        # Fitting function
        history = model.fit(X_train, y_train, batch_size=batch_size, class_weight=class_weight, epochs=epochs, 
                            steps_per_epoch=steps_per_epoch,
                            validation_data=(X_test, y_test), verbose=2, callbacks=[es, mcp, csv_logger, tensorboard_callback])

        return model, history, logpath, csv_path, model_name


In [54]:
def create_train_LSTM_model(X_train, X_test, y_train, y_test, class_weight, epochs, 
                       steps_per_epoch=None, layer_size=200, batch_size=2160, optimizer=Nadam(learning_rate=0.00001), metrics=['accuracy'], activity_regularizer=l1(0.0001), prefix="", layers=1, data_path=''):

        # Get dimensions for model input layer from input array
        data_width = X_train.shape[1]
        data_depth = X_train.shape[2]
        shape = (data_width,data_depth)

            
        # Create Sequential Model 
        model = Sequential()
        model.add(InputLayer(input_shape=shape))
        model.add(LSTM(int(layer_size), activity_regularizer=activity_regularizer, return_sequences=True))
        model.add(Dropout(.4))
        model.add(Flatten())     
        model.add(Dense(5))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=metrics)
    
        model.build()
        model.summary()
        
        # Name model with logging based on training time and date and metrics to track
        model_name=f"Single_LSTM_size_{LSTM_size}_{prefix}_"
        filepath = f"{model_name}"+"_weights.auc:{auc}-{categorical_accuracy:.6f}--val-{val_auc}-{val_categorical_accuracy:.6f}.h5"
        
        # Keras model checkpoint function
        mcp = ModelCheckpoint(f"./Checkpoints/{filepath}", 
                              monitor='val_loss', verbose=1, 
                              save_best_only=True, save_weights_only=False, mode='min')       
    
        # Early stopping parameters              
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                    patience=5, min_delta=0.001,restore_best_weights=True)
    
        # Tensorboard logging variables    
        log_dir = f"./Logs/{model_name}"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        logpath=f"./Logs/"   
        csv_path=f"{model_name}_model.csv"
        csv_logger=CSVLogger(logpath+csv_path)
        
        # Model fitting call
        history = model.fit(X_train, y_train, batch_size=batch_size, class_weight=class_weight, epochs=epochs, 
                            steps_per_epoch=steps_per_epoch,
                            validation_data=(X_test, y_test), verbose=2, callbacks=[es, mcp, csv_logger, tensorboard_callback])

        return model, history, logpath, csv_path, model_name

In [55]:
def create_train_BiLSTM_model(X_train, X_test, y_train, y_test, class_weight, epochs, 
                       steps_per_epoch=None, layer_size=200, batch_size=2160, optimizer=Nadam(learning_rate=0.00001), metrics=['accuracy'], activity_regularizer=l1(0.0001), prefix="", layers=1, data_path=''):
        
        # Get dimensions for model input layer from input array
        data_width = X_train.shape[1]
        data_depth = X_train.shape[2]
        shape = (data_width,data_depth)

        # Create Sequential Model 
        model = Sequential()
        model.add(InputLayer(input_shape=shape))
        forward_layer = LSTM(int(layer_size/factor), activity_regularizer=activity_regularizer, return_sequences=True)
        backward_layer = LSTM(int(layer_size/factor), activity_regularizer=activity_regularizer, return_sequences=True,
                              go_backwards=True)
        model.add(Bidirectional(forward_layer, backward_layer=backward_layer))
        model.add(Dropout(.4))
        model.add(Flatten())     
        model.add(Dense(5))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=metrics)
        
        model.build()
        model.summary()
        
        # Name model with logging based on training time and date and metrics to track
        model_name=f"BiLSTM_size_{LSTM_size}_{prefix}_"
        filepath = f"{model_name}"+"_weights-auc-{auc}-{categorical_accuracy:.6f}--val-{val_auc}-{val_categorical_accuracy:.6f}.h5"

        # Keras model checkpoint function
        mcp = ModelCheckpoint(f"./Checkpoints/{filepath}", 
                              monitor='val_loss', verbose=1, 
                              save_best_only=True, save_weights_only=False, mode='min')     
    
        # Early stopping parameters                                            
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                    patience=5, min_delta=0.001,restore_best_weights=True)

        # Tensorboard logging variables        
        log_dir = f"./Logs/{model_name}"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        logpath=f"./Logs/"
        csv_path=f"{model_name}_model.csv"
        csv_logger=CSVLogger(logpath+csv_path)
        
        # Model fitting call
        history = model.fit(X_train, y_train, batch_size=batch_size, class_weight=class_weight, epochs=epochs, 
                            steps_per_epoch=steps_per_epoch,
                            validation_data=(X_test, y_test), verbose=2, callbacks=[es, mcp, csv_logger, tensorboard_callback])

        return model, history, logpath, csv_path, model_name

In [56]:
def create_train_flat_BiLSTM_model(X_train, X_test, y_train, y_test, class_weight, epochs, 
                       steps_per_epoch=None, layer_size=200, batch_size=2160, optimizer=Nadam(learning_rate=0.00001), metrics=['accuracy'], activity_regularizer=l1(0.0001), prefix="", layers=3, data_path=''):

        # Get dimensions for model input layer from input array
        data_width = X_train.shape[1]
        data_depth = X_train.shape[2]
        shape = (data_width,data_depth)
            
        # Create Sequential Model 
        # Model creation with optional stacking of Bi-LSTM/Dropout pairs
        model = Sequential()
        model.add(InputLayer(input_shape=shape))
        model.add(Flatten())           
        model.add(RepeatVector(1)) 

        # Each Bi-LSTM in cascade gets smaller by a factor of 2
        if layers in range(1,5):
            forward_layer = LSTM(int(LSTM_size), return_sequences=True)
            backward_layer = LSTM(int(LSTM_size), return_sequences=True,
                                  go_backwards=True)
            model.add(Bidirectional(forward_layer, backward_layer=backward_layer))
            model.add(RepeatVector(1)) 
            model.add(Dropout(.4))

            factor = 2
            if layers in range (2,5):
              forward_layer = LSTM(int(LSTM_size/factor), return_sequences=True)
              backward_layer = LSTM(int(LSTM_size/factor), return_sequences=True,
                                    go_backwards=True)
              model.add(Bidirectional(forward_layer, backward_layer=backward_layer))
              model.add(Dropout(.4))
              model.add(RepeatVector(1)) 

              factor = 4
              if layers in range (3,5):
                forward_layer = LSTM(int(LSTM_size/factor), return_sequences=True)
                backward_layer = LSTM(int(LSTM_size/factor), return_sequences=True,
                                      go_backwards=True)
                model.add(Bidirectional(forward_layer, backward_layer=backward_layer))
                model.add(Dropout(.4))
                model.add(RepeatVector(1)) 

                factor = 8
                if layers in range (4,5):
                    forward_layer = LSTM(int(LSTM_size/factor), return_sequences=True)
                    backward_layer = LSTM(int(LSTM_size/factor), return_sequences=True,
                                          go_backwards=True)
                    model.add(Bidirectional(forward_layer, backward_layer=backward_layer))
                    model.add(Dropout(.4))
                    model.add(RepeatVector(1)) 
                
        model.add(Flatten())     
        model.add(Dense(5))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=METRICS)
        
        model.build()
        model.summary()
                
        # Name model with logging based on training time and date and metrics to track
        model_name=f"flat_LSTM_size_{LSTM_size}_{prefix}_"
        filepath = f"{model_name}"+"_weights-auc-{auc}-{categorical_accuracy:.6f}--val-{val_auc}-{val_categorical_accuracy:.6f}.h5"

        # Keras model checkpoint function
        mcp = ModelCheckpoint(f"./Checkpoints/{filepath}", 
                              monitor='val_loss', verbose=1, 
                              save_best_only=True, save_weights_only=False, mode='min')      
    
        # Early stopping parameters                                            
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                    patience=5, min_delta=0.001,restore_best_weights=True)

        # Tensorboard logging variables        
        log_dir = f"./Logs/{model_name}"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        logpath=f"./Logs/"
        csv_path=f"{model_name}_flat_model.csv"
        csv_logger=CSVLogger(logpath+csv_path)
        
        # Model fitting call
        history = model.fit(X_train, y_train, batch_size=batch_size, class_weight=class_weight, epochs=epochs, 
                            steps_per_epoch=steps_per_epoch,
                            validation_data=(X_test, y_test), verbose=2, callbacks=[es, mcp, csv_logger, tensorboard_callback])


        return model, history, logpath, csv_path, model_name

## Training Bookkeeping Functions

In [57]:
def save_model_report_seq(model, model_history, ctrls=0, run=2, x=[], y=[], X_train_seq=[], X_val_seq=[], y_train_seq=[], y_val_seq=[], X_test_seq=[], y_test_seq=[], model_name='test',i=1,model_num=8,data_path='',logpath='', csvpath='',savepath='./Results/'):  
    
    # Print model name to output
    print(model_name)
    
    # Assess Confusion Matrices with limited states for full-saline cohorts 
    if ctrls==1:
        num_classes=3
    else:
        num_classes=5

    # Name files based on dataset 
    if ctrls==0:
        charpath=f'Kainic Acid'
    elif ctrls==1:
        charpath=f'Control'
    elif ctrls==2:
        charpath='AllData'


    # Make output directories for models, confusion matrices, and softmax confidence outputs
    if os.path.exists(savepath)==False:
        os.mkdir(savepath)
    if os.path.exists(f'{savepath}Models/')==False:
        os.mkdir(f'{savepath}Models/')
    if os.path.exists(f'{savepath}Softmax/')==False:
        os.mkdir(f'{savepath}Softmax/')

    # Codes for generating and naming confusion matrices
    codes=np.array(['Wake','NREM','REM','Seizure','Post-Ictal'])

    # Save hdf5 model within output folder
    if os.path.exists(f'{savepath}Models/{model_name}.h5')==False:
       model.save(f'{savepath}Models/{model_name}.h5')  

    # Save Model History dictionary 
    if model_history!=None:
        with open(f'{savepath}{model_name}{model_num}historydict','wb') as file_pi:
            pickle.dump(model_history.history, file_pi)
          
        shutil.copyfile(logpath+csvpath, f'{savepath}{csvpath}')
        
    # Reconstitute entire dataset for evaluation
    x_new=np.concatenate((X_test_seq,X_val_seq, X_train_seq))
    y_new_1hot=np.concatenate((y_test_seq,y_val_seq, y_train_seq))
      
    # With model report text file open, generate report text
    with open(f'{savepath}Report_{model_name}.txt','w') as f:
        # Iterate over train/test/val and combined datasets 
        # Create Matrices/Reports for all datasets in train/test/val, as well as overall statistics
        for selection in range(0,4):
            gc.collect()
            names=['Saline-KA Holdout Testing Dataset', 'Saline Control Validation Dataset', 'Training Dataset', 'Training, Validation, and Testing Overall']
            X_check, y_check = [[X_test_seq, y_test_seq], [X_val_seq, y_val_seq],[X_train_seq, y_train_seq], [x_new, y_new_1hot]][selection]

            # Predict scores 
            y_pred=model.predict(X_check, batch_size=2160)
            # Save softmax results
            y_pred.tofile(f'{savepath}Softmax/results_{model_name}.csv', sep=',')
            
            # Get codes from softmax results
            y_pred_codes=np.argmax(y_pred,1)
            y_test_codes=np.argmax(y_check,1)
            
            # Print classification reports to Report txt
            print(f'\n {names[selection]} \n', file=f)
            print(classification_report(y_test_codes, y_pred_codes), file=f)

            # Save classification report as dataframe to excel 
            dict1 = classification_report(y_test_codes, y_pred_codes, output_dict=True)
            df = pd.DataFrame(data=dict1)
            df = (df.T)
            print (df)
            dataset_name=names[selection]
            df.to_excel(f'{savepath}{dataset_name}{model_name}.xlsx')

            # Generate normalized confusion matrix 
            cm=confusion_matrix(y_test_codes,y_pred_codes,normalize='true')

            # Sanity check that all states in dataset are accounted for in results
            print(f'unique ground truth labels: {unique(y_test_codes)}')
            print(f'unique predicted labels: {unique(y_pred_codes)}')
    
            # Display confusion matrix with whichever dataset has more labels in it - otherwise it will not plot 
            if len(unique(y_pred_codes))>len(unique(y_test_codes)):
              disp=ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=codes[unique(y_pred_codes)])
            else:
              disp=ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=codes[unique(y_test_codes)])
    
            disp.plot()

            # Plot and save confusion matrix as .png and .svg for publication
            print(f'{names[selection]}')
            plt.title(f'Confusion Matrix for {names[selection]}')
            plt.xlabel('Predicted')
            plt.ylabel('True')
            plt.savefig(f'{savepath}{model_name}_{names[selection]}_CM_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.png', dpi=700)
            plt.savefig(f'{savepath}{model_name}_{names[selection]}_CM_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.svg', dpi=700)
    
    
            plt.close()
            gc.collect()

In [58]:
def save_model_report_svg(model, model_history, ctrls=0, run=2, x=[], y=[], X_train_seq=[], X_val_seq=[], y_train_seq=[], y_val_seq=[], X_test_seq=[], y_test_seq=[], model_name='test',i=1,model_num=8,data_path='', savepath='./Results/',textsize = 15):  

    # Same function as save_model_report_seq, but without saving of history dicts and models, just for confusion matrix output
    
    # Print model name to output
    print(model_name)

    # Assess Confusion Matrices with limited states for full-saline cohorts 
    if ctrls==1:
        num_classes=3
    else:
        num_classes=5

    # Name files based on dataset 
    if ctrls==0:
        charpath=f'Kainic Acid'
    elif ctrls==1:
        charpath=f'Control'
    elif ctrls==2:
        charpath='AllData'
    
    # Make output directories for confusion matrices and softmax confidence outputs
    if os.path.exists(savepath)==False:
        os.mkdir(savepath)

    # Codes for generating and naming confusion matrices
    codes=np.array(['Wake','NREM','REM','Seizure','Post-Ictal'])


    # Reconstitute entire dataset for evaluation
    x_new=np.concatenate((X_test_seq,X_val_seq, X_train_seq))
    y_new_1hot=np.concatenate((y_test_seq,y_val_seq, y_train_seq))
  
    # With model report text file open, generate report text
    with open(f'{savepath}Report_{model_name}.txt','w') as f:
        # Iterate over train/test/val and combined datasets 
        # Create Matrices/Reports for all datasets in train/test/val, as well as overall statistics
        for selection in range(0,4):
            gc.collect()
            names=['Saline-KA Holdout Testing Dataset', 'Saline Control Validation Dataset', 'Training Dataset', 'Training, Validation, and Testing Overall']
            X_check, y_check = [[X_test_seq, y_test_seq], [X_val_seq, y_val_seq],[X_train_seq, y_train_seq], [x_new, y_new_1hot]][selection]
            
            # Predict scores 
            y_pred=model.predict(X_check, batch_size=2160)
            # Save softmax results
            y_pred.tofile(f'{savepath}Softmax/results_{model_name}.csv', sep=',')
    
            # Get codes from softmax results
            y_pred_codes=np.argmax(y_pred,1)
            y_test_codes=np.argmax(y_check,1)

            # Print classification reports to Report txt
            print(f'\n {names[selection]} \n', file=f)
            print(classification_report(y_test_codes, y_pred_codes), file=f)
            
            # Save classification report as dataframe to excel 
            dict1 = classification_report(y_test_codes, y_pred_codes, output_dict=True)
            df = pd.DataFrame(data=dict1)
            df = (df.T)
            print (df)
            dataset_name=names[selection]
            df.to_excel(f'{savepath}{dataset_name}{model_name}.xlsx')
            
            # Generate normalized confusion matrix 
            cm=confusion_matrix(y_test_codes,y_pred_codes,normalize='true')

            # Sanity check that all states in dataset are accounted for in results
            print(f'unique ground truth labels: {unique(y_test_codes)}')
            print(f'unique predicted labels: {unique(y_pred_codes)}')
    
            # Display confusion matrix with whichever dataset has more labels in it - otherwise it will not plot 
            if len(unique(y_pred_codes))>len(unique(y_test_codes)):
              disp=ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=codes[unique(y_pred_codes)])
            else:
              disp=ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=codes[unique(y_test_codes)])
    
            
            # Plot and save confusion matrix as .png and .svg for publication
            disp.plot()
            print(f'{names[selection]}')
            plt.title(f'Confusion Matrix for {names[selection]}')
            plt.xlabel('Predicted')
            plt.ylabel('True')
            plt.savefig(f'{savepath}{model_name}_{names[selection]}_CM_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.png', dpi=700)
            plt.savefig(f'{savepath}{model_name}_{names[selection]}_CM_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.svg', dpi=700)
    
    
            plt.close()
            gc.collect()

In [59]:
def print_conf_mat(y_test_codes, y_pred_codes, legend, output_name):

      
        labelmax=max(max(y_test_codes),max(y_pred_codes))
        print(labelmax)
        print(codes[0:labelmax+1])

        disp=ConfusionMatrixDisplay.from_predictions(y_test_codes,y_pred_codes, labels=[0,1,2,3,4], normalize='true', display_labels=codes)

        plt.title(f'{legend}')
        plt.xlabel('Predicted')
        plt.ylabel('True')

        plt.savefig(f'{output_name}_ConfMat.png', dpi=700)
        plt.show()

## Post-Training Scoring Functions

In [60]:
def score_data(paths, errors, load_dropped_data=0, held_dates=[], scored=0):
    
    # Print model summary to output
    model.summary()

    # Get all paths
    data_path,path_Fourier_scored,path_Fourier_unscored,path_output,path_conf=paths

    # Evaluate based on designated category of existing scoring
    # 1 saves classifier scores for previously-scored data
    if scored==1:
      strpart='trainset_classifier'
      path_Fourier=path_Fourier_scored
    # 2 saves manual expert scores for previously-scored data
    elif scored==2:
      strpart='trainset_expert'
      path_Fourier=path_Fourier_scored
    # 0 saves classifier scores for previously-unscored data
    else:
      strpart='classifier_prev_un'
      path_Fourier=path_Fourier_unscored

    # Establish counting variables for tracking data processing
    %matplotlib inline
    score=0
    good=0

    # Generate path for scoring subtype
    path_csv=f'{path_output}'+f'csv_{strpart}_scored/'
    for path_iterate in [path_conf,path_csv]:
        if os.path.isdir(path_iterate) == False:
          os.mkdir(path_iterate)
    print(path_csv)

    print(path_Fourier)
    fourier_files = [f for f in listdir(path_Fourier) if isfile(join(path_Fourier, f))]
    fourier_files = [f for f in fourier_files if "x_ffnorm" in f]   # find all x arrays in fourier_files

    # If load_dropped_data is 1, only load dates within held_dates, if not then exclude held_dates
    if load_dropped_data==1:
        print('loading dropped data')
        fourier_files_pruned=[]
        for d in held_dates:
            fourier_files=[f for f in fourier_files if d in f]
        fourier_files_pruned=fourier_files
    else:
        for d in held_dates:
            fourier_files = [f for f in fourier_files if d not in f]
        fourier_files_pruned=fourier_files

    test_length=len(fourier_files_pruned)  
    print(test_length)

    # Create display handles and variables for loop tracking
    dh1 = display(f'Items left: {test_length}',display_id=True)
    dh = display('Loading...',display_id=True)
    y_done=0
    x_done=0
    global_counter = 0
    
    window=3
    shift=window*2
    max_feats=100
    classes=5

    for item in (sorted(fourier_files_pruned)):
        # Reset file exclusion flag
        exclude_file = 0

        # Update displays with current remaining files
        test_length-=1
        dh1.update(f'Items left: {test_length}')

        # If file is already scored, set exclusion flag to 1
        item = item.replace("x_ffnorm", "y_ffnorm")
        if os.path.isfile(f'{path_csv}{item}_{strpart}_scored.csv')==True:
          exclude_file=1
        item = item.replace("y_ffnorm", "x_ffnorm")

        # Skip file if already scored
        if exclude_file==0:
            # Load 
            current_file_x, x_done, item = load_file(path_Fourier, item, scored)
            
            if scored > 0:
                current_file_y = np.load(f"{path_Fourier}{item}")
            y_done=1

            X_score3=np.zeros((len(current_file_x)-shift,window*2+1,max_feats))

            for i in range(window,len(current_file_x)-window):
                X_score3[i-window]=current_file_x[i-window:i+window+1,0:max_feats]
          
            if x_done == 1 and y_done == 1:
                global_counter+=1

                num_classes=5

                if scored in [0,1]:
                    y_pred=model.predict(X_score3, 2160, verbose=0)

                    y_pred_codes=np.argmax(y_pred,1)
                    np.save(f'{path_conf}conf_{item}',y_pred)
                    y_pred_codes.tofile(f'{path_csv}{item}_{strpart}_scored.csv', sep = ',')

                else:
                  y_pred_codes=current_file_y-1
                  y_pred_codes.tofile(f'{path_csv}{item}_{strpart}_scored.csv', sep = ',')

                dh.update(f'loaded x and y for {item}')

                if global_counter%50==0:
                    print(global_counter)
                x_done=0
                y_done=0



        else:
            print(f'{item} excluded')
            excl=0

## Post-Training Evaluation

In [73]:
def R_and_K_evaluation(y_pred, violations_total):
    y_pred_scores = y_pred
    # Store existing scores 
    y_pred_scores_baseline = y_pred_scores

    # Rechtshaffen and Kales Critera
    violations=np.zeros((3))
    rem_violations=0
    nrem_violations=0
    wake_violations=0
    
    
    for idx in range(0,len(y_pred_scores)-1):
        if idx>0:
            # REM Must Follow NREM
            # "If REM is preceded by Wake, score as Wake"
            if y_pred_scores[idx]==2 and y_pred_scores[idx-1]==0:
                rem_violations+=1

                y_pred_scores[idx]=0

            # In order to score NREM, there must be 2 consecutive epochs
            # or lone NREM is scored as the previous epoch
            if y_pred_scores[idx]==1:
                if y_pred_scores[idx-1]!=1 and y_pred_scores[idx+1]!=1:
                    nrem_violations+=1

                    y_pred_scores[idx]=y_pred_scores[idx-1]


    violations[0]=wake_violations
    violations[1]=rem_violations
    violations[2]=nrem_violations
    violations_total[0]=violations[0]+violations_total[0]
    violations_total[1]=violations[1]+violations_total[1]
    violations_total[2]=violations[2]+violations_total[2]

    agreement = y_pred_scores == y_pred_scores_baseline
    agree_pct=(len(y_pred_scores)-np.sum(violations))/len(y_pred_scores)

    return agree_pct, agreement, violations_total

In [74]:
def load_file(path_Fourier, item, scored):
    # Handles 4 and 20 second files agnostically

    # Load and standard-scale FFT arrays
    current_file_x = StandardScaler().fit_transform(np.load(f"{path_Fourier}{item}"))

    # Determine expansion factor of RMS EMG
    length_factor = int(len(current_file_x)/2160)

    # If RMS EMG not binned in 20 1-second bins
    if  length_factor != 1:
        
        last_rms_index=20-int(20/length_factor)
        # Repeat the RMS EMG 20/epoch_length times
        for index in range(len(current_file_x)):
            current_rms=current_file_x[index,-20:-last_rms_index]
            current_file_x[index,-20:]=np.repeat(current_rms,length_factor)

    x_done=1

    return current_file_x, x_done, item

# Load Data for Training

In [75]:
# Datasets selected for Train/Test Split via this block

old_cohort = ['566']  # not trained on animals with radically different signal
held_cohorts = ['200316','566', '569', ' NPM569 ', 
                'NPM570 ', ' NPM572 ', ' NPM574 ', ' NPM579 ',
                'NPM573-576', ' NPM580 ',' NPM614 '] # Wild type or control mice with lack of baseline or classifier-breaking scoring issues 

wild_types=['NPM648','NPM652', 'NPM656'] # WT KA mouse cohort identifiers
wild_types_saline=['NPM644'] # WT saline mouse identifiers - no quotes ensures entire 644 cohort is loaded
saline = [' NPM592 ', ' NPM596 ',' NPM604 '] # Saline 


recent = [' NPM645 ',' NPM646 ',' NPM647 ', 'NPM656-659',
          ' NPM661 ',' NPM662 ', ' NPM663 ', ' NPM664 ',' NPM665 ',' NPM666 ', 
          ' NPM667 ', ' NPM668 '] # Validation set of most recent animals
          # Mixed batch of KA and Saline, VGAT Cre and WT

extra_train = [' NPM609 ',' NPM644 ']
              # ' NPM664 '] 
# random saline animals (609,644) and recent-KA animals (664) to add to training dataset

held_dates = ['200103','200104','200105','200106',
       '200107','200108','200109',
       '200301','200302','200303',
       '200614','200615','200616',
       '200719','200720','200721',
       '200809','200810','200811',
       '201101','200102','201103',
       '201213','201214','201215',
       '210131','200201','210202',
       '210321','200322','210323',
       '200620','200621','200622']
       # all dates from baseline recordings of KA mice were ignored for training, 
       # due to issues sorting by condition vs. mouse vs. date



path=path_Fourier_scored
fourier_files = [f for f in listdir(path) if isfile(join(path, f))]
fourier_files = [f for f in fourier_files if "x_ffnorm" in f]   # find all x arrays in fourier_files

non_train=saline+wild_types_saline+recent+held_cohorts+wild_types

print(non_train)
files_held=[]
files_held=[f for d in non_train for f in fourier_files if d in f] # gather list of all non-training files

train_files=[]
train_files=[f for f in fourier_files if f not in files_held] # gather list of all training files
np.savetxt(data_path+'training_files.txt', train_files,delimiter=" ", fmt="%s") 

x1,y1=load_Fourier_xy(path, train_files, standardize=True, train_recording_epoch_count=train_recording_epoch_count, viable_scores=viable_scores)


val_files=[]
val_files=[f for d in saline for f in fourier_files if d in f] # gather list of all saline VGAT Cre files for test dataset
np.savetxt(data_path+'validation_files.txt', val_files,delimiter=" ", fmt="%s") 

x2,y2=load_Fourier_xy(path, val_files, standardize=True, train_recording_epoch_count=train_recording_epoch_count, viable_scores=viable_scores)

test_files=[]
test_files=[f for d in recent+wild_types for f in fourier_files if d in f] # gather list of all validation files marked by 'recent' variable
np.savetxt(data_path+'testing_files.txt', test_files,delimiter=" ", fmt="%s") 

x3,y3=load_Fourier_xy(path, test_files, standardize=True, train_recording_epoch_count=train_recording_epoch_count, viable_scores=viable_scores)


[' NPM592 ', ' NPM596 ', ' NPM604 ', 'NPM644', ' NPM645 ', ' NPM646 ', ' NPM647 ', 'NPM656-659', ' NPM661 ', ' NPM662 ', ' NPM663 ', ' NPM664 ', ' NPM665 ', ' NPM666 ', ' NPM667 ', ' NPM668 ', '200316', '566', '569', ' NPM569 ', 'NPM570 ', ' NPM572 ', ' NPM574 ', ' NPM579 ', 'NPM573-576', ' NPM580 ', ' NPM614 ', 'NPM648', 'NPM652', 'NPM656']
707


'Items left: 0'

'loaded x and y for x_ffnorm NPM642 210223 190404_058 NPM639-642.npy'

50
100
150
200
250
300
350
400
450
500
550
600
650
700
129


'Items left: 0'

'loaded x and y for x_ffnorm NPM604 200707 190222_013 NPM604-607.npy'

50
100
340


'Items left: 0'

'loaded x and y for x_ffnorm NPM668 210713 190351_058 NPM665-668.npy'

50
100
150
200
250
300


In [27]:
# List all wild-type animals
wild_type_list=[f for d in wild_types+wild_types_saline for f in fourier_files if d in f]
names=[]
for i in wild_type_list:
  names.append(i[5:11])
print(len(unique(names)))

# List all saline animals
saline_list=[f for d in saline+wild_types_saline+[' NPM609 '] for f in fourier_files if d in f]
names=[]
for i in saline_list:
  names.append(i[5:11])
print(len(unique(names)))

# Get all input fourier files 
path=path_Fourier_scored
fourier_files = [f for f in listdir(path) if isfile(join(path, f))]
fourier_files = [f for f in fourier_files if "x_ffnorm" in f]

non_train=saline+wild_types_saline+recent+held_cohorts+wild_types
print(non_train)
 
# Generate list of all non-training files
files_held=[]
files_held=[f for d in non_train for f in fourier_files if d in f]

# Generate list of all training dataset files
train_files=[]
train_files=[f for f in fourier_files if f not in files_held] 
names=[]

for i in train_files:
  names.append(i[9:15])
train_names=unique(names)
np.savetxt(data_path+'training_animals.txt', train_names,delimiter=" ", fmt="%s") 
print(*train_names, sep = "\n")
print(len(unique(train_names)))

# Generate list of all validation dataset files
names=[]
validation_files=[f for d in saline for f in fourier_files if d in f] # gather list of all saline VGAT Cre files for test dataset
for i in validation_files:
    names.append(i[9:15])
    
validation_names=unique(names)
np.savetxt(data_path+'validation_animals.txt', validation_names,delimiter=" ", fmt="%s") 
print(*validation_names, sep = "\n")
print(len(unique(validation_names)))


names=[]
testing_files=[f for d in recent+wild_types for f in fourier_files if d in f] 
for i in testing_files:
    names.append(i[9:15])

testing_names=unique(names)
np.savetxt(data_path+'testing_animals.txt', testing_names,delimiter=" ", fmt="%s") 
print(*testing_names, sep = "\n")
print(len(unique(testing_names)))

print((set(testing_names)-set(validation_names)))

1
1
[' NPM592 ', ' NPM596 ', ' NPM604 ', 'NPM644', ' NPM645 ', ' NPM646 ', ' NPM647 ', 'NPM656-659', ' NPM661 ', ' NPM662 ', ' NPM663 ', ' NPM664 ', ' NPM665 ', ' NPM666 ', ' NPM667 ', ' NPM668 ', '200316', '566', '569', ' NPM569 ', 'NPM570 ', ' NPM572 ', ' NPM574 ', ' NPM579 ', 'NPM573-576', ' NPM580 ', ' NPM614 ', 'NPM648', 'NPM652', 'NPM656']
NPM577
NPM589
NPM590
NPM593
NPM595
NPM605
NPM606
NPM607
NPM608
NPM609
NPM610
NPM611
NPM612
NPM613
NPM615
NPM617
NPM620
NPM621
NPM622
NPM623
NPM624
NPM625
NPM626
NPM628
NPM629
NPM632
NPM633
NPM634
NPM635
NPM636
NPM637
NPM638
NPM639
NPM640
NPM642
35
NPM592
NPM596
NPM604
3
NPM645
NPM646
NPM647
NPM648
NPM649
NPM650
NPM652
NPM653
NPM654
NPM655
NPM656
NPM658
NPM659
NPM661
NPM663
NPM664
NPM665
NPM666
NPM667
NPM668
20
{'NPM667', 'NPM666', 'NPM653', 'NPM656', 'NPM658', 'NPM661', 'NPM652', 'NPM664', 'NPM650', 'NPM648', 'NPM663', 'NPM646', 'NPM668', 'NPM665', 'NPM654', 'NPM649', 'NPM655', 'NPM647', 'NPM645', 'NPM659'}


In [79]:
for datasets in [train_files,val_files,test_files]:

    names=[]
    counts=np.zeros((70))
    for i in datasets:
        names.append(i[9:15])
    
    unique_array=[]
    ind=0
    for name in names:
        if name not in unique_array:
            unique_array.append(name)
            ind=unique_array.index(name)
            counts[ind]+=1

    
        else:
            counts[ind]+=1
            
    print(unique_array)
    print(len(unique_array))
    counts
    bookkeeping=np.ndarray((len(unique_array),2),dtype='object')
    print(bookkeeping.shape)
    for i in range(len(unique_array)):
        bookkeeping[i,0]=unique_array[i]
        bookkeeping[i,1]=counts[i]

    print(bookkeeping)

['NPM577', 'NPM589', 'NPM590', 'NPM593', 'NPM595', 'NPM605', 'NPM606', 'NPM607', 'NPM608', 'NPM609', 'NPM610', 'NPM611', 'NPM612', 'NPM613', 'NPM615', 'NPM617', 'NPM620', 'NPM621', 'NPM622', 'NPM623', 'NPM624', 'NPM625', 'NPM626', 'NPM628', 'NPM629', 'NPM632', 'NPM633', 'NPM634', 'NPM635', 'NPM636', 'NPM637', 'NPM638', 'NPM639', 'NPM640', 'NPM642']
35
(35, 2)
[['NPM577' 11.0]
 ['NPM589' 46.0]
 ['NPM590' 54.0]
 ['NPM593' 40.0]
 ['NPM595' 40.0]
 ['NPM605' 17.0]
 ['NPM606' 17.0]
 ['NPM607' 17.0]
 ['NPM608' 17.0]
 ['NPM609' 54.0]
 ['NPM610' 17.0]
 ['NPM611' 17.0]
 ['NPM612' 16.0]
 ['NPM613' 16.0]
 ['NPM615' 17.0]
 ['NPM617' 18.0]
 ['NPM620' 16.0]
 ['NPM621' 16.0]
 ['NPM622' 16.0]
 ['NPM623' 4.0]
 ['NPM624' 16.0]
 ['NPM625' 16.0]
 ['NPM626' 16.0]
 ['NPM628' 16.0]
 ['NPM629' 16.0]
 ['NPM632' 16.0]
 ['NPM633' 16.0]
 ['NPM634' 16.0]
 ['NPM635' 16.0]
 ['NPM636' 16.0]
 ['NPM637' 16.0]
 ['NPM638' 16.0]
 ['NPM639' 16.0]
 ['NPM640' 17.0]
 ['NPM642' 16.0]]
['NPM592', 'NPM596', 'NPM604']
3
(3, 2)
[['

# Model Creation

In [80]:
# Create array of codes to link numeric labels in y variables to state labels

codes=[None]*5
codes[0]='Wake'
codes[1]='NREM' 
codes[2]='REM'
codes[3]='Seizure'
codes[4]='Post-Ictal'

In [81]:
keras.backend.clear_session()

In [82]:
# Run before re-running training

X_train_seq=None
X_val_seq=None
X_test_seq=None
model=None
tf.keras.backend.clear_session()
tf.compat.v1.reset_default_graph()
# tf.reset_default_graph()

# Purges excess garbage data from RAM if training needs to be re-run
for i in range(0,10):
  gc.collect()

# Pre-Process X and Y Variables


In [85]:
# These arrays correspond to lists of indices in the feature vector
# for each of our channels

# Mask containing only the Delta/Theta Ratio and RMS variables
mask_DTRMS=np.concatenate(([12,19,32,39,52,59,72,79],np.arange(80,100)))

# Mask containing only the FFT variables and RMS - no statistical parameters
mask_FFT_only = []
for i in [0,1,2,3]:
  ind1=(i*20)+6
  ind2=(i*20)+13
  mask_FFT_only=np.concatenate((mask_FFT_only,np.arange(ind1,ind2)))
mask_FFT_only=np.concatenate((mask_FFT_only,np.arange(80,100)))
mask_FFT_only=[int(i) for i in mask_FFT_only]

# Masks for each individual channel's full feature sets
mask_ECoG=np.arange(0,20)
mask_EMG=np.arange(20,40)
mask_HPCL=np.arange(40,60)
mask_HPCR=np.arange(60,80)
mask_RMS=np.arange(80,100)
mask_FullFeats=np.arange(0,100)

# Designate feature mask to use for preparing variables
# This construction would send only HPCL and HPCR channels to training
# mask = np.concatenate((mask_HPCL, mask_HPCR)) # mask_FullFeats lets all 100 features through

mask=mask_FullFeats

In [86]:
# Convert ys to one-hot labels, store x variables to 
# counteract future in-place edits - makes it easier to re-run notebook


X_train=x1
y_train=to_categorical(y1, num_classes=5)

X_val=x2
y_val=to_categorical(y2, num_classes=5)

X_test=x3
y_test=to_categorical(y3, num_classes=5)

X_train_masked=X_train[:,mask]
X_val_masked=X_val[:,mask]
X_test_masked=X_test[:,mask]

#  Grid Search Block

## Full Deep Learning Grid Search

In [ ]:
Channel_Masks=[[mask_ECoG, 'ECoG_only'],
               [np.concatenate((mask_HPCL,mask_HPCR)), '2xHPC'],
               [np.concatenate((mask_EMG,mask_HPCL,mask_HPCR,mask_RMS)), 'no_ECoG'],
               [np.concatenate((mask_EMG,mask_RMS)), 'EMG_RMS_only' ],
               [np.concatenate((mask_ECoG,mask_EMG,mask_RMS)), 'noHPC']]

Feature_Masks = [[mask_FullFeats, 'FullFeats'],
                 [mask_FFT_only, 'FFT only'],
                 [mask_DTRMS,'DTR+RMS only']]

# Generate training variables which are common between models
batch_size, metrics, optimizer, activity_regularizer = common_training_parameters()

# Number of epochs to train 
epochs=20

for window_length in [3,2,1,0]:
  # Test all four windowing variants 
  # Grid Search Iteration Count in This Loop: 4 Combinations
  for masks in Feature_Masks:
    # Test all three Feature Vectors 
    # Grid Search Iteration Count in This Loop: 12 Combinations
      
    X_train_masked=X_train[:,masks[0]]
    X_val_masked=X_val[:,masks[0]]
    X_test_masked=X_test[:,masks[0]]

    # Mask name for training logs
    submodel=masks[1]
    print(submodel)  

    # Maximum number of features determined by last dimension in shape of masked array
    max_feats=X_train_masked.shape[1]

    # Generate windowed sequences from full epoch arrays
      
    # Number of time points on either side of scored epoch to use for input variables...
    # determined by window length.  i.e. window_length 3 leads to an input 7 epochs long
    X_train_seq=generate_sequences(X_train_masked, windows=window_length, x_or_y='X', max_feats=max_feats)
    X_val_seq=generate_sequences(X_val_masked, windows=window_length, x_or_y='X', max_feats=max_feats)
    X_test_seq=generate_sequences(X_test_masked, windows=window_length, x_or_y='X', max_feats=max_feats)

    y_train_seq=generate_sequences(y_train, windows=window_length, x_or_y='Y', max_feats=max_feats)
    y_val_seq=generate_sequences(y_val, windows=window_length, x_or_y='Y', max_feats=max_feats)
    y_test_seq=generate_sequences(y_test, windows=window_length, x_or_y='Y', max_feats=max_feats)
      
    # Generate class weight array based on logarithmic modification of sk-learn class weighting
    class_weight=norm_sklearn_classweight(y_train_seq, mu=False)
    print(class_weight)

    print(X_train_seq.shape)

    for neuron_num in [200,100,50]:
        # Test Three Initial-Layer Neuron Counts
        # Grid Search Iteration Count in This Loop: 36 Combinations

        
        # Test All Above Over 4 Types of Models: 144 Total Model Variants
        # Dense Neural Net Model training
        model, model_history, logpath, csvpath, model_name = create_train_Dense_model(X_train_seq, X_val_seq, y_train_seq, y_val_seq, class_weight=class_weight, 
                              epochs=epochs, steps_per_epoch=None, layer_size=neuron_num, batch_size=batch_size, optimizer=optimizer, metrics=metrics, activity_regularizer=activity_regularizer, prefix="", layers=1, data_path=data_path)
        gc.collect()
        # Dense Neural Net Model Report
        savepath='./Results_GPU/Dense/'
        save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_val_seq=X_val_seq, y_train_seq=y_train_seq, y_val_seq=y_val_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                            model_name=model_name+f'_{submodel}_win{window_length}', 
                            model_num=neuron_num, data_path=data_path, logpath=logpath, csvpath=csvpath, savepath=savepath)
        gc.collect()
        
        # LSTM Model training
        model, model_history, logpath, csvpath, model_name = create_train_LSTM_model(X_train_seq, X_val_seq, y_train_seq, y_val_seq, class_weight=class_weight, 
                              epochs=epochs, steps_per_epoch=None, layer_size=neuron_num, batch_size=batch_size, optimizer=optimizer, metrics=metrics, activity_regularizer=activity_regularizer, prefix="", layers=1, data_path=data_path)
        gc.collect()
        # LSTM Model Report
        savepath='./Results_GPU/LSTM/'
        save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_val_seq=X_val_seq, y_train_seq=y_train_seq, y_val_seq=y_val_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                            model_name=model_name+f'_{submodel}_win{window_length}_', 
                            model_num=neuron_num, data_path=data_path, logpath=logpath, csvpath=csvpath, savepath=savepath)
        gc.collect()
        
        # BiLSTM Model Training
        model, model_history, logpath, csvpath, model_name = create_train_BiLSTM_model(X_train_seq, X_val_seq, y_train_seq, y_val_seq, class_weight=class_weight, 
                              epochs=epochs, steps_per_epoch=None, layer_size=neuron_num, batch_size=batch_size, optimizer=optimizer, metrics=metrics, activity_regularizer=activity_regularizer, prefix="", layers=1, data_path=data_path)
        gc.collect()
        # BiLSTM Model Report
        savepath='./Results_GPU/BiLSTM/'
        save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_val_seq=X_val_seq, y_train_seq=y_train_seq, y_val_seq=y_val_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                            model_name=model_name+f'single-layer_{submodel}_win{window_length}_', 
                            model_num=neuron_num, data_path=data_path, logpath=logpath, csvpath=csvpath, savepath=savepath)
      

        # Stacked BiLSTM Model Training
        model, model_history, logpath, csvpath, model_name = create_train_BiLSTM_model(X_train_seq, X_val_seq, y_train_seq, y_val_seq, class_weight=class_weight, 
                              epochs=epochs, steps_per_epoch=None, layer_size=neuron_num, batch_size=batch_size, optimizer=optimizer, metrics=metrics, activity_regularizer=activity_regularizer, prefix="", layers=4, data_path=data_path)
        gc.collect()
        # Stacked BiLSTM Model Report
        savepath='./Results_GPU/Quad_BiLSTM/'
        save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_val_seq=X_val_seq, y_train_seq=y_train_seq, y_val_seq=y_val_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                            model_name=model_name+f'quad-layer_{submodel}_win{window_length}_', 
                            model_num=neuron_num, data_path=data_path, logpath=logpath, csvpath=csvpath, savepath=savepath)

## Evaluate Top Models to 60 Epochs

In [63]:
Channel_Masks=[[mask_ECoG, 'ECoG_only'],
               [np.concatenate((mask_HPCL,mask_HPCR)), '2xHPC'],
               [mask_HPCL, 'HPCL'],
               [np.concatenate((mask_EMG,mask_HPCL,mask_HPCR,mask_RMS)), 'no_ECoG'],
               [np.concatenate((mask_EMG,mask_RMS)), 'EMG_RMS_only' ],
               [np.concatenate((mask_ECoG,mask_EMG,mask_RMS)), 'noHPC']]

Feature_Masks = [[mask_DTRMS,'DTR+RMS only'],
  [mask_FFT_only, 'FFT only'],
  [mask_FullFeats, 'FullFeats']]

# Only perform this loop over the window length of 3 and Full Feature mask 

# Generate training variables which are common between models
batch_size, metrics, optimizer, activity_regularizer = common_training_parameters()

# Number of epochs to train 
epochs=20

for window_length in [3]:

    for masks in [Feature_Masks[2]]:
      
        X_train_masked=X_train[:,masks[0]]
        X_val_masked=X_val[:,masks[0]]
        X_test_masked=X_test[:,masks[0]]
    
        # Mask name for training logs
        submodel=masks[1]
        print(submodel)  
    
        # Maximum number of features determined by last dimension in shape of masked array
        max_feats=X_train_masked.shape[1]
    
        # Generate windowed sequences from full epoch arrays
          
        # Number of time points on either side of scored epoch to use for input variables...
        # determined by window length.  i.e. window_length 3 leads to an input 7 epochs long
        X_train_seq=generate_sequences(X_train_masked, windows=window_length, x_or_y='X', max_feats=max_feats)
        X_val_seq=generate_sequences(X_val_masked, windows=window_length, x_or_y='X', max_feats=max_feats)
        X_test_seq=generate_sequences(X_test_masked, windows=window_length, x_or_y='X', max_feats=max_feats)
    
        y_train_seq=generate_sequences(y_train, windows=window_length, x_or_y='Y', max_feats=max_feats)
        y_val_seq=generate_sequences(y_val, windows=window_length, x_or_y='Y', max_feats=max_feats)
        y_test_seq=generate_sequences(y_test, windows=window_length, x_or_y='Y', max_feats=max_feats)
    
        # Generate class weight array based on logarithmic modification of sk-learn class weighting
        class_weight=norm_sklearn_classweight(y_train_seq, mu=False)
        print(class_weight)

        print(X_train_seq.shape)
    
        for neruon_num in [200]:
            
            # LSTM Model Training
            model, model_history, logpath, csvpath, model_name = create_train_LSTM_model(X_train_seq, X_val_seq, y_train_seq, y_val_seq, class_weight=class_weight, 
                              epochs=epochs, steps_per_epoch=None, layer_size=neuron_num, batch_size=batch_size, optimizer=optimizer, metrics=metrics, activity_regularizer=activity_regularizer, prefix="", layers=1, data_path=data_path)
            gc.collect()
            # LSTM Model Report
            savepath='./Results_GPU/60_Epochs/LSTM/'
            save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_val_seq=X_val_seq, y_train_seq=y_train_seq, y_val_seq=y_val_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                                model_name=model_name+f'_{submodel}_win{window_length}_', 
                                model_num=nn, data_path=data_path, logpath=logpath, csvpath=csvpath, savepath=savepath)
            gc.collect()

            
            # BiLSTM Model Training
            model, model_history, logpath, csvpath, model_name = create_train_BiLSTM_model(X_train_seq, X_val_seq, y_train_seq, y_val_seq, class_weight=class_weight, 
                              epochs=epochs, steps_per_epoch=None, layer_size=neuron_num, batch_size=batch_size, optimizer=optimizer, metrics=metrics, activity_regularizer=activity_regularizer, prefix="", layers=1, data_path=data_path)
            
            # BiLSTM Model Report
            savepath='./Results_GPU/60_Epochs/BiLSTM/'
            save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_val_seq=X_val_seq, y_train_seq=y_train_seq, y_val_seq=y_val_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                                model_name=model_name+f'single-layer_{submodel}_win{window_length}_', 
                                model_num=nn, data_path=data_path, logpath=logpath, csvpath=csvpath, savepath=savepath)
            gc.collect()

            
            # BiLSTM Model Report
            model, model_history, logpath, csvpath, model_name = create_train_BiLSTM_model(X_train_seq, X_val_seq, y_train_seq, y_val_seq, class_weight=class_weight, 
                              epochs=epochs, steps_per_epoch=None, layer_size=neuron_num, batch_size=batch_size, optimizer=optimizer, metrics=metrics, activity_regularizer=activity_regularizer, prefix="", layers=4, data_path=data_path)
            gc.collect()
            # BiLSTM Model Report
            savepath='./Results_GPU/60_Epochs/Quad_BiLSTM/'
            save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_val_seq=X_val_seq, y_train_seq=y_train_seq, y_val_seq=y_val_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                                model_name=model_name+f'quad-layer_{submodel}_win{window_length}_', 
                                model_num=nn, data_path=data_path, logpath=logpath, csvpath=csvpath, savepath=savepath)
            gc.collect()


FullFeats
(1527114, 7, 100)
[3.85603689e-01 4.84991322e-01 2.94539563e+00 1.01133377e+03
 2.34219939e+02]
{0: 1.0, 1: 1.229320869330697, 2: 3.033188298565099, 3: 8.871970458237705, 4: 7.409205733125976}


C:\Users\BHARVE4\AppData\Local\Temp\ipykernel_6672\3115110011.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  j = int(np.where(classes==i)[0])


Model: "sequential_155"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_195 (Bidirect  (None, 7, 400)           481600    
 ional)                                                          
                                                                 
 dropout_233 (Dropout)       (None, 7, 400)            0         
                                                                 
 flatten_155 (Flatten)       (None, 2800)              0         
                                                                 
 dense_194 (Dense)           (None, 5)                 14005     
                                                                 
 activation_155 (Activation)  (None, 5)                0         
                                                                 
Total params: 495,605
Trainable params: 495,605
Non-trainable params: 0
______________________________________________

## Train Interpretable Machine Learning Models via Channel-Dropping

In [23]:
mask_ECoG=np.arange(0,20)
mask_EMG=np.arange(20,40)
mask_HPCL=np.arange(40,60)
mask_HPCR=np.arange(60,80)
mask_RMS=np.arange(80,100)

Channel_Masks=[[mask_ECoG, 'ECoG_only'],
               [np.concatenate((mask_HPCL,mask_HPCR)), '2xHPC'],
               [np.concatenate((mask_EMG,mask_HPCL,mask_HPCR,mask_RMS)), 'no_ECoG'],
               [np.concatenate((mask_EMG,mask_RMS)), 'EMG_RMS_only' ],
               [np.concatenate((mask_ECoG,mask_EMG,mask_RMS)), 'noHPC'],
               [mask_HPCL, 'HPCL'],]

Feature_Masks = [[mask_DTRMS,'DTR+RMS only'],
                [mask_FFT_only, 'FFT only'],
                [mask_FullFeats, 'FullFeats']]

# Generate training variables which are common between models
batch_size, metrics, optimizer, activity_regularizer = common_training_parameters()

# Number of epochs to train 
epochs=20

for window_length in [3]:
    for masks in Channel_Masks:
        X_train_masked=X_train[:,masks[0]]
        X_val_masked=X_val[:,masks[0]]
        X_test_masked=X_test[:,masks[0]]
    
        # Mask name for training logs
        submodel=masks[1]
        print(submodel)  
    
        # Maximum number of features determined by last dimension in shape of masked array
        max_feats=X_train_masked.shape[1]
    
        # Number of time points on either side of scored epoch to use for input variables...
        # determined by window length.  i.e. window_length 3 leads to an input 7 epochs long
        X_train_seq=generate_sequences(X_train_masked, windows=window_length, x_or_y='X', max_feats=max_feats)
        X_val_seq=generate_sequences(X_val_masked, windows=window_length, x_or_y='X', max_feats=max_feats)
        X_test_seq=generate_sequences(X_test_masked, windows=window_length, x_or_y='X', max_feats=max_feats)
    
        y_train_seq=generate_sequences(y_train, windows=window_length, x_or_y='Y', max_feats=max_feats)
        y_val_seq=generate_sequences(y_val, windows=window_length, x_or_y='Y', max_feats=max_feats)
        y_test_seq=generate_sequences(y_test, windows=window_length, x_or_y='Y', max_feats=max_feats)
        
        # Generate class weight array based on logarithmic modification of sk-learn class weighting
        class_weight=norm_sklearn_classweight(y_train_seq, mu=False)
        print(class_weight)
        
        print(X_train_seq.shape)
    
        for nn in [200]:
            # Train Interpretable BiLSTM Model
            gc.collect()
            model, model_history, logpath, csvpath, model_name = create_train_BiLSTM_model(X_train_seq, X_val_seq, y_train_seq, y_val_seq, class_weight=class_weight, 
                        epochs=epochs, layer_size=neuron_num, batch_size=batch_size, optimizer=optimizer, metrics=metrics, activity_regularizer=activity_regularizer, prefix="", layers=1, data_path=data_path)
            gc.collect()
            
            # Save Interpretable BiLSTM Report
            savepath='./Results_GPU/Interpretable/'
            
            save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, 
                        X_train_seq=X_train_seq, X_val_seq=X_val_seq, y_train_seq=y_train_seq, y_val_seq=y_val_seq, 
                        X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                        model_name=model_name+f'single-layer_{submodel}_win{window_length}_', 
                        model_num=nn, data_path=data_path, logpath=logpath, csvpath=csvpath, savepath=savepath)
            
            save_model_report_svg(model, model_history, ctrls=2, run=2, x=None, y=None, 
                        X_train_seq=X_train_seq, X_val_seq=X_val_seq, y_train_seq=y_train_seq, y_val_seq=y_val_seq, 
                        X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                        model_name=model_name+f'_{submodel}_win{window_length}', 
                        model_num=nn, data_path=data_path, savepath=savepath, textsize=15)



NameError: name 'mask_DTRMS' is not defined

## Generate Classification Matrices for Previous Models

In [88]:
# Generate Classification Matrices for full-featured model 

# Max X arrays
mask=mask_FullFeats 
submodel="FullFeats"
X_train_masked=X_train[:,mask]
X_val_masked=X_val[:,mask]
X_test_masked=X_test[:,mask]

# Generate Windowed Sequences for X and Y arrays
window_length=3 

X_train_seq=generate_sequences(X_train_masked, windows=window_length, x_or_y='X', max_feats=100)
X_val_seq=generate_sequences(X_val_masked, windows=window_length, x_or_y='X', max_feats=100)
X_test_seq=generate_sequences(X_test_masked, windows=window_length, x_or_y='X', max_feats=100)

y_train_seq=generate_sequences(y_train, windows=window_length, x_or_y='Y', max_feats=100)
y_val_seq=generate_sequences(y_val, windows=window_length, x_or_y='Y', max_feats=100)
y_test_seq=generate_sequences(y_test, windows=window_length, x_or_y='Y', max_feats=100)

# Load model
model_path=data_path+'Results_GPU/Final Model/Final Model BiLSTM200.h5'
model=load_model(model_path)
model.summary()
model_name='BiLSTM_200'

savepath=data_path+'Results_GPU/Final Model/'

# Save Model Report
save_model_report_svg(model, model_history, ctrls=2, run=2, x=None, y=None, 
                      X_train_seq=X_train_seq, X_val_seq=X_val_seq, y_train_seq=y_train_seq, 
                      y_val_seq=y_val_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                      model_name=model_name+f'_{submodel}_win{window_length}', 
                      model_num=nn, data_path=data_path, savepath=savepath, textsize=15)

Model: "sequential_155"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_195 (Bidirect  (None, 7, 400)           481600    
 ional)                                                          
                                                                 
 dropout_233 (Dropout)       (None, 7, 400)            0         
                                                                 
 flatten_155 (Flatten)       (None, 2800)              0         
                                                                 
 dense_194 (Dense)           (None, 5)                 14005     
                                                                 
 activation_155 (Activation)  (None, 5)                0         
                                                                 
Total params: 495,605
Trainable params: 495,605
Non-trainable params: 0
______________________________________________

# Train SWISC Model

In [ ]:
masks=[mask_FullFeats, 'FullFeats']

print(f'\n{masks[1]}')
X_train_masked=X_train[:,masks[0]]
X_val_masked=X_val[:,masks[0]]
X_test_masked=X_test[:,masks[0]]

submodel=masks[1]

window length=3
    
# Maximum number of features determined by last dimension in shape of masked array
max_feats=X_train_masked.shape[1]

# Number of time points on either side of scored epoch to use for input variables...
# determined by window length.  i.e. window_length 3 leads to an input 7 epochs long
X_train_seq=generate_sequences(X_train_masked, windows=window_length, x_or_y='X', max_feats=max_feats)
X_val_seq=generate_sequences(X_val_masked, windows=window_length, x_or_y='X', max_feats=max_feats)
X_test_seq=generate_sequences(X_test_masked, windows=window_length, x_or_y='X', max_feats=max_feats)

y_train_seq=generate_sequences(y_train, windows=window_length, x_or_y='Y', max_feats=max_feats)
y_val_seq=generate_sequences(y_val, windows=window_length, x_or_y='Y', max_feats=max_feats)
y_test_seq=generate_sequences(y_test, windows=window_length, x_or_y='Y', max_feats=max_feats)

print(X_train_seq.shape)

# Generate training variables which are common between models
batch_size, metrics, optimizer, activity_regularizer = common_training_parameters()

# Number of epochs to train 
epochs=20

# Generate class weight array based on logarithmic modification of sk-learn class weighting
class_weight=norm_sklearn_classweight(y_train_seq, mu=False)
print(class_weight)

for neuron_num in [200]:

  gc.collect()
    
 model, model_history, logpath, csvpath, model_name = create_train_BiLSTM_model(X_train_seq, X_val_seq, y_train_seq, y_val_seq, class_weight=class_weight, 
                              epochs=epochs, steps_per_epoch=None, layer_size=neuron_num, batch_size=batch_size, optimizer=optimizer, metrics=metrics, activity_regularizer=activity_regularizer, prefix="", layers=1, data_path=data_path)
  gc.collect()

  savepath='./Results_GPU/60_Epochs/BiLSTM/'

  save_model_report_seq(model, model_history, ctrls=2, run=2, x=None, y=None, X_train_seq=X_train_seq, X_val_seq=X_val_seq, y_train_seq=y_train_seq, y_val_seq=y_val_seq, X_test_seq=X_test_seq, y_test_seq=y_test_seq,
                        model_name=model_name+f'single-layer_{submodel}_win{window_length}_', 
                        model_num=neuron_num, data_path=data_path, logpath=logpath, csvpath=csvpath, savepath=savepath)


# Final Scoring of Data

In [2]:
model_path=data_path+'Results_GPU/Final Model/Final Model BiLSTM200.h5'
model=load_model(modelpath)
model.compile()
model.summary()

In [ ]:
held_cohorts = []

# held_dates=held_cohorts+controls+held_dates
held_dates=[]

scoretypes=[[0,'unscored'],[1,'class_scored'],[2,'expert_scored']]

errors=[]
paths=(data_path,path_Fourier_scored,path_Fourier_unscored,path_output,path_conf)

for flag, types in scoretypes:
  print(f'Generating CSVs for {types}')
  score_data(paths, errors, load_dropped_data=0, held_dates=[], scored=flag)

# CSV Exports 

In [115]:
# Sort through files and iterate over unscored files, classifier scores for previously-scored files, and manual scores for previously-scored files
# Copy these files to a directory where they are split by animal and named by category 

for sel in [0,1,2]:

    filepath = [path_csv,path_holes,path_expert][sel]
    filelist=os.listdir(filepath)
    print(len(filelist))
    count=0
    if count==0:
        dh1 = display(f'Items left: {len(filelist)}',display_id=True)

    for item in sorted(filelist)[::1]:
        # Skip if file is not .csv (i.e. if it is a folder path)
        if '.csv' in item:
            count+=1    

            # Update counter in intervals of 50
            if count%50==0 or (len(filelist)-count)==0:
                dh1.update(f'Items left: {len(filelist)-count}')
    
            ## Extract Mouse Name *specific to Pedersen Lab*
            ind_name = item.find('NPM')
            name = item[ind_name:ind_name+6]
      
            path_out=f'{mouse_sort_csvs}{name}/'

            # Make mouse-specific folder
            if os.path.exists(path_out)==0:
                os.mkdir(path_out)

            # Copy file to mouse-specific folder
            if os.path.isfile(f'{path_out}{item}')==0:
                shutil.copy(f'{filepath}{item}', f'{path_out}{item}')

4565


'Items left: 0'

1569


'Items left: 0'

1569


'Items left: 0'

# Rechtschaffen & Kales Scoring Layer

In [ ]:
# Specific animal numbers for Pedersen Lab KA cohorts, used for scoring evaluation and output
KA_list=[576,577,578,580,  589,590,591,  593,594,595,  605,606,607, 608,610,611,612,
         613,615,616,617,  618,619,620,621,622,623,624,625,626,627,628,629,630,631,
         632,633,634,636,637,638,639,640,642,648,649,650,651,652,653,
         654,655,656,657,658,659,661,662,663,664,665,666,667,668,688,690,691,693,695,696,697]

Excl_KA=[568,573,575,577,578,591,594,616,618,619,623,627,630,631,642,658,659,660,662,665]
Excl_no_SE=[575,589,590,605,606,607,608,610]
Excl_death=[565,568,573,577,578,591,594,616,618,619,623,627,630,631,662,665]
Excl_no_Epilepsy=[611,615,617,625,634,640,667,668]
Excl_no_Epilepsy=[]
Excluded=[630,631,641,643,648,651,655,657,658,662,663]
Excl_WT=[614,644,645,646,647,648,649,650,651,652,653,654,655,656,657]
Excl_early=[572,571,570,569,567,566,564]
    Sal_pre_cannula=[569,570,571,572,574]

Excluded_paper=Excl_KA+Excluded+Excl_WT+Excl_no_SE+Excl_no_Epilepsy+Excl_death
Excluded_agreement=Excl_death+KA_pre_cannula+Excl_KA+Sal_pre_cannula
Excluded=Excl_KA+Excluded+Excl_no_SE+Excl_no_Epilepsy+Excl_death+Excl_early

In [ ]:
# Set figure size to 20 x 10
plt.rcParams['figure.figsize'] = [20, 10]
mplstyle.use('fast')

# List all mouse folders
folderlist=sorted(os.listdir(mouse_sort_csvs))
folderlist=[f for f in sorted(os.listdir(mouse_sort_csvs)) if '.csv' not in f]
print(folderlist)

# Flags to use either expert scores or classifier scores
use_expert=0
skipped_type='expert_scored'
use_type='classifier'

score_label='Trained or Tested Data'

slope_sum=np.zeros((4,1))
avg_slope=np.zeros((4,1))

# Display handles for R&K agreement
dh1=display(f'agreement: ',display_id=True)
dh2=display(f'violation: ',display_id=True)

# Tracking variables for use in for loop
global_agreement=0
n_animals=0
global_count=0
violations_total=np.zeros((3))

# Iterate over all animals, to evaluate by treatment group
for Controls in [False,True]:
    for folder in folderlist[::-1]:

        # Reset flags for each animal's folder
        KA_flag=False
        skip=0
        mouse_folder = f'{mouse_sort_csvs}{folder}/'
        filelist=sorted(os.listdir(mouse_folder))

        filelist = [f for f in filelist if use_type in f]   
        
        # Check if animal is a KA-treated animal
        for KA in KA_list:
            if f'{KA}' in folder:
                KA_flag=True

        # Check if animal is in exclusion list
        for Excl in Excluded:
            if f'{Excl}' in folder:
                skip=1

        # If animal matches treatment group flag and is not excluded then
        if KA_flag!=Controls and skip==0:
            if len(filelist)>0:
                # Iterate over all files
                for item in filelist[::1]:
                    path = open(f'{mouse_folder}{item}')

                    # Load only classifier-scored files
                    if 'expert_scored' not in item:
                        y_pred = np.loadtxt(path, delimiter=",",dtype='int')   

                    # Run R&K evaluation algorithm and log agreements
                    agree_pct, agreement, violations_total = R_and_K_evaluation(y_pred, violations_total):
                    
                    # Add agreement percent to running tally
                    global_agreement+=agree_pct            

                    global_count+=1
                    # Calculate global agreement, and update display with this along with type of R&K violations
                    dh1.update(f'{global_agreement/global_count}')
                    dh2.update(f'violations: {np.sum(violations_total)}')

# Print total file count
print(global_count)
# Print percent of R&K errors
print(1-global_agreement/global_count)
# Print total number of R&K errors
print(violations_total)
# Print relative percentage of types of R&K errors
print(violations_total/(np.sum(violations_total)))

# Plot Agreement Between Expert and Classifier

## Functions

In [133]:
del init_displays

In [134]:
import matplotlib.style as mplstyle


def get_figure_params(mouse_sort_csvs,use_expert):
    folderlist=sorted(os.listdir(mouse_sort_csvs))
    folderlist=[f for f in sorted(os.listdir(mouse_sort_csvs)) if '.csv' not in f]

    print(folderlist)
    KA_pre_cannula=[554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,573,576]

    KA_list_paper=[576,577,578, 589,590,591,  593,594,595,  605,606,607, 608,610,611,612,
         613,615,616,617,  618,619,620,621,622,623,624,625,626,627,628,629,630,631,
         632,633,634,635,636,637,638,639,640,642,648,649,650,651,652,653,
         654,655,656,657,658,659,661,662,663,664,665,666,667,668,688,690,691,693,695,696,697]

    # returned values
    if use_expert==1:
      skipped_type='classifier'
      use_type='expert_scored'

      score_label='Expert Scoring'
    else:
      skipped_type='expert_scored'
      use_type='classifier'

      score_label='Trained or Tested Data'
    
    return folderlist, use_type, skipped_type, KA_list_paper, KA_pre_cannula
    
    
# def init_displays():
#     dh_no=display(f'file: ',display_id=True)
#     dh1=display(f'agreement: ',display_id=True)
#     dh_maxagree=display(f'max agreement: ',display_id=True)
#     dh_minagree=display(f'min agreement: ',display_id=True)
#     dh_stddev=display(f'std dev of agreement: ',display_id=True)

#     dh_conf=display(f'confidence: ',display_id=True)
    
#     return dh_no, dh1, dh_maxagree, dh_minagree, dh_stddev, dh_conf



def init_vars():
    global_agreement=0
    global_confidence=0
    max_agree=0
    min_agree=100
    n_animals=0
    
    return global_agreement, global_confidence, max_agree, min_agree, n_animals



def update_vars(array, array2, confidence_gradient, min_agree, max_agree, global_agreement, agreement_list, std_dev, global_confidence, count, global_count):
    agreement = array==array2[3:-3]

    agreement_score=np.zeros(len(agreement))
    agreement_score[agreement==True]=1
    agreement_score[agreement==False]=0

    maxlen=len(array)
    agree_pct=len(agreement[agreement==True])/(maxlen-6)
    agreement_list.append(agree_pct)
    if len(agreement_list)>1:
        std_dev=np.std(agreement_list)

    if agree_pct>max_agree:
            max_agree=agree_pct
    if agree_pct<min_agree:
            min_agree=agree_pct

    if np.isnan(confidence_gradient.mean())==False:
                    global_confidence=global_confidence+confidence_gradient.mean()

    count+=1
    global_count+=1
    
    global_agreement=global_agreement+agree_pct
    
    return agreement, min_agree, max_agree, global_agreement, agreement_list, std_dev, global_confidence, count, global_count



    
def update_displays(min_agree, max_agree, global_agreement, std_dev, global_confidence, count, global_count):
    dh_no.update(f'file: {global_count}')
    dh1.update(f'agreement: {global_agreement/global_count}')
    dh_maxagree.update(f'max agreement: {max_agree}')
    dh_minagree.update(f'min agreement: {min_agree}')
    dh_stddev.update(f'std dev of agreement: {std_dev}')
    if np.isnan(confidence_gradient.mean())==False:
            dh_conf.update(f'confidence: {global_confidence/global_count}')
    
    return dh_no, dh1, dh_maxagree, dh_minagree, dh_stddev, dh_conf

def load_expert_and_classifier(mouse_folder, item):
    path = open(f'{mouse_folder}{item}')

    if 'expert_scored' in item:
        expert_array = np.loadtxt(path, delimiter=",",dtype='float')
        expert_array = expert_array.astype('int')
        classifier_filename=item.replace("expert", "classifier")
     
    else:
        array2 = np.loadtxt(path, delimiter=",",dtype='int')
        item2=item

    path = open(f'{mouse_folder}{classifier_filename}')
    classifier_array = np.loadtxt(path, delimiter=",",dtype='int')     

    idx_name=item.find('npy_')
    conf_item='conf_'+item[:idx_name-1]+'.npy'
    conf = np.load(path_conf+conf_item)

    return expert_array, classifier_array, conf

def plot_agreement_figure(array, array2, agreement, agreement_score, confidence_gradient):
                x1 = np.arange(0,len(array2))

                plt.figure(figsize=(width,height), dpi=2000)
                ax1=plt.subplot(2,4, (1,3))    
# Ax1
                
                ax1.bar(x1[3:maxlen-3], agreement_score,color='k', width=1)
                ax1.bar(np.ma.masked_where(~agreement, x1[3:maxlen-3]), np.ma.masked_where(agreement, agreement_score+1), color='grey', width=1)

                # ax1.title.set_text(f'Sleep Record Scoring Comparision')

                black_patch = mpatches.Patch(color='k', label='Agreement')
                grey_patch = mpatches.Patch(color='grey', label='Difference')

                ax1.legend(handles=[black_patch, grey_patch], bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0.)
                ax1.text(x_legend_offset, .5, f'Classifier and \n Expert \n Agreement', horizontalalignment='center', 
                                 verticalalignment='center', transform=ax1.transAxes)
                ax1.set_xticks(range(0,maxlen+int(maxlen/12),int(maxlen/12)))

                if Z_time=='night':
                    time_vect=[str(x)+":00" for x in list(range(19,24))+list(range(0,8))]
                else:
                    time_vect=[str(x)+":00" for x in range(7,20)]


                ax1.set_xticklabels(time_vect)

# Ax2

                ax2=plt.subplot(2,4,(5,7))    

                codes=['PI', 'Sz', 'W', 'N', 'R']
                types=max(array)
                # print(f'max legend = {types}')         
                for i in range(len(array)):
                        if confidence_gradient[i]>0:
                                confidence=confidence_gradient[i][0]
                                c=round(confidence,2)
                        else:
                                c=.1
                        ax2.vlines(x=i,ymin=3,ymax=3.5, color=[0,c,0])

                array[array==4]=-2
                array[array==3]=-1

                array2[array2==4]=-2
                array2[array2==3]=-1
                
                ax2.plot(x1[3:maxlen-3], array, 'red',linewidth=.3)
                ax2.plot(x1[3:maxlen-3], array2[3:maxlen-3], color='grey',linewidth=.3)

# agreement plotting

                grey_patch = mpatches.Patch(color='grey', label='Expert Scoring')
                black_patch = mpatches.Patch(color='k', label='Classifier Scoring')
                ax2.legend(handles=[black_patch, grey_patch], bbox_to_anchor=(1.02, 1),
                                                 loc='upper left', borderaxespad=0.)

                ax2.text(x_legend_offset, .5, f'Hypnogram', horizontalalignment='center', verticalalignment='center', transform=ax2.transAxes)
                ax2.set_xticks(range(0,maxlen+int(maxlen/12),int(maxlen/12)))
                ax2.set_xticklabels(time_vect)
                ax2.set_xlabel("Time")
                if Z_time=='night':
                    ax2.set_facecolor("lightgray")
# y tick assignment            
                
                ticklist=ax2.get_yticks()
                # print(ticklist)
                low_legend=min([min(array), min(array2)])
                # print(f'low legend = {low_legend}')                

                high_legend=max([max(array), max(array2)])
                # print(f'high legend = {high_legend}')



                ax2.set_yticks(range(-2,3))
                ax2.set_yticklabels(labels=codes)
                ax2.invert_yaxis()                


# Ax3
                ax3=plt.subplot(2,4, (4))
                ax3.pie([agree_pct,1-agree_pct], colors=['k', 'grey'],autopct='%1.1f%%', pctdistance=1.7, center=(0,-5))
                pie = ax3.get_position()
                pie.y0 = pie.y0 - 0.05
                pie.y1 = pie.y1 - 0.05

                ax3.set_position(pie)
                plt.show()

In [135]:
import matplotlib.style as mplstyle


def get_figure_params(mouse_sort_csvs,use_expert):
    KA_pre_cannula=[554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,573,576]

    KA_list_paper=[576,577,578, 589,590,591,  593,594,595,  605,606,607, 608,610,611,612,
             613,615,616,617,  618,619,620,621,622,623,624,625,626,627,628,629,630,631,
             632,633,634,635,636,637,638,639,640,642,648,649,650,651,652,653,
             654,655,656,657,658,659,661,662,663,664,665,666,667,668,688,690,691,693,695,696,697]

    Excl_KA=[568,573,575,578,580,591,594,616,618,619,623,627,630,631,642,658,659,660,662,665]
    Excl_no_SE=[575,589,590,605,606,607,608,610]
    # Excl_no_SE=[]
    Excl_death=[565,568,573,578,591,594,616,618,619,623,627,630,631,662,665]
    Excl_no_Epilepsy=[611,615,617,625,634,640,667,668]
    Excl_no_Epilepsy=[]
    Excluded=[630,631,641,643,648,651,655,657,658,662,663,577]
    Excl_WT=[614,644,645,646,647,648,649,650,651,652,653,654,655,656,657]
    Sal_pre_cannula=[569,570,571,572,574]

    # returned value
    Excluded_paper=Excl_KA+Excluded+Excl_WT+Excl_no_SE+Excl_no_Epilepsy+Excl_death
    Excluded_agreement=Excl_death+KA_pre_cannula+Excl_KA+Sal_pre_cannula
    folderlist=sorted(os.listdir(mouse_sort_csvs))
    # returned value
    folderlist=[f for f in sorted(os.listdir(mouse_sort_csvs)) if '.csv' not in f]

    print(folderlist)

    
    # returned values
    if use_expert==1:
      skipped_type='classifier'
      use_type='expert_scored'

      score_label='Expert Scoring'
    else:
      skipped_type='expert_scored'
      use_type='classifier'

      score_label='Trained or Tested Data'
    
    return folderlist, use_type, skipped_type, Excluded_paper, Excluded_agreement, KA_list_paper, KA_pre_cannula
    
    
def init_displays():
    dh_no=display(f'file: ',display_id=True)
    dh1=display(f'agreement: ',display_id=True)
    dh_maxagree=display(f'max agreement: ',display_id=True)
    dh_minagree=display(f'min agreement: ',display_id=True)
    dh_stddev=display(f'std dev of agreement: {std_dev}',display_id=True)

    dh_conf=display(f'confidence: ',display_id=True)
    
    return dh_no, dh1, dh_maxagree, dh_minagree, dh_stddev, dh_conf



def init_vars():
    global_agreement=0
    global_confidence=0
    max_agree=0
    min_agree=100
    n_animals=0
    
    return global_agreement, global_confidence, max_agree, min_agree, n_animals



def update_vars(array, array2, confidence_gradient, min_agree, max_agree, global_agreement, agreement_list, std_dev, global_confidence, count, global_count):
    agreement = array==array2[3:-3]

    agreement_score=np.zeros(len(agreement))
    agreement_score[agreement==True]=1
    agreement_score[agreement==False]=0

    maxlen=len(array)
    agree_pct=len(agreement[agreement==True])/(maxlen-6)
    agreement_list.append(agree_pct)
    if len(agreement_list)>1:
        std_dev=np.std(agreement_list)
    # print(len(agreement[agreement==False]))
    # print(f'agreement:{agree_pct}')
    # print(np.corrcoef( array, array2[3:-3])[0,1])

    # print(f'mean confidence: {confidence_gradient.mean()}')

    if agree_pct>max_agree:
            max_agree=agree_pct
    if agree_pct<min_agree:
            min_agree=agree_pct

    # print(f'agreement:{agree_pct}')

    # print(confidence_gradient[array==0])

#                 np.set_printoptions(threshold=sys.maxsize)

#                 display(pd.DataFrame([emg_trace, array]).transpose())
#                 np.set_printoptions(threshold=False)

    if np.isnan(confidence_gradient.mean())==False:
                    global_confidence=global_confidence+confidence_gradient.mean()

    count+=1
    global_count+=1
    
    global_agreement=global_agreement+agree_pct
    
    return agreement, min_agree, max_agree, global_agreement, agreement_list, std_dev, global_confidence, count, global_count



    
def update_displays(min_agree, max_agree, global_agreement, std_dev, global_confidence, count, global_count):
    dh_no.update(f'file: {global_count}')
    dh1.update(f'agreement: {global_agreement/global_count}')
    dh_maxagree.update(f'max agreement: {max_agree}')
    dh_minagree.update(f'min agreement: {min_agree}')
    dh_stddev.update(f'std dev of agreement: {std_dev}')
    if np.isnan(confidence_gradient.mean())==False:
            dh_conf.update(f'confidence: {global_confidence/global_count}')
    
    return dh_no, dh1, dh_maxagree, dh_minagree, dh_stddev, dh_conf




def plot_agreement_figure(array, array2, agreement, agreement_score, confidence_gradient):
                x1 = np.arange(0,len(array2))

                plt.figure(figsize=(width,height), dpi=2000)
                ax1=plt.subplot(2,4, (1,3))    
# Ax1
                
                ax1.bar(x1[3:maxlen-3], agreement_score,color='k', width=1)
                ax1.bar(np.ma.masked_where(~agreement, x1[3:maxlen-3]), np.ma.masked_where(agreement, agreement_score+1), color='grey', width=1)

                # ax1.title.set_text(f'Sleep Record Scoring Comparision')

                black_patch = mpatches.Patch(color='k', label='Agreement')
                grey_patch = mpatches.Patch(color='grey', label='Difference')

                ax1.legend(handles=[black_patch, grey_patch], bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0.)
                ax1.text(x_legend_offset, .5, f'Classifier and \n Expert \n Agreement', horizontalalignment='center', 
                                 verticalalignment='center', transform=ax1.transAxes)
                ax1.set_xticks(range(0,maxlen+int(maxlen/12),int(maxlen/12)))

                if Z_time=='night':
                    time_vect=[str(x)+":00" for x in list(range(19,24))+list(range(0,8))]
                else:
                    time_vect=[str(x)+":00" for x in range(7,20)]


                ax1.set_xticklabels(time_vect)

# Ax2

                ax2=plt.subplot(2,4,(5,7))    

                codes=['PI', 'Sz', 'W', 'N', 'R']
                types=max(array)
                # print(f'max legend = {types}')         
                for i in range(len(array)):
                        if confidence_gradient[i]>0:
                                confidence=confidence_gradient[i][0]
                                c=round(confidence,2)
                        else:
                                c=.1
                        ax2.vlines(x=i,ymin=3,ymax=3.5, color=[0,c,0])

                array[array==4]=-2
                array[array==3]=-1

                array2[array2==4]=-2
                array2[array2==3]=-1
                
                ax2.plot(x1[3:maxlen-3], array, 'red',linewidth=.3)
                ax2.plot(x1[3:maxlen-3], array2[3:maxlen-3], color='grey',linewidth=.3)

# agreement plotting

                grey_patch = mpatches.Patch(color='grey', label='Expert Scoring')
                black_patch = mpatches.Patch(color='k', label='Classifier Scoring')
                ax2.legend(handles=[black_patch, grey_patch], bbox_to_anchor=(1.02, 1),
                                                 loc='upper left', borderaxespad=0.)

                ax2.text(x_legend_offset, .5, f'Hypnogram', horizontalalignment='center', verticalalignment='center', transform=ax2.transAxes)
                ax2.set_xticks(range(0,maxlen+int(maxlen/12),int(maxlen/12)))
                ax2.set_xticklabels(time_vect)
                ax2.set_xlabel("Time")
                if Z_time=='night':
                    ax2.set_facecolor("lightgray")
# y tick assignment            
                
                ticklist=ax2.get_yticks()
                # print(ticklist)
                low_legend=min([min(array), min(array2)])
                # print(f'low legend = {low_legend}')                

                high_legend=max([max(array), max(array2)])
                # print(f'high legend = {high_legend}')



                ax2.set_yticks(range(-2,3))
                ax2.set_yticklabels(labels=codes)
                ax2.invert_yaxis()                


# Ax3
                ax3=plt.subplot(2,4, (4))
                ax3.pie([agree_pct,1-agree_pct], colors=['k', 'grey'],autopct='%1.1f%%', pctdistance=1.7, center=(0,-5))
                pie = ax3.get_position()
                pie.y0 = pie.y0 - 0.05
                pie.y1 = pie.y1 - 0.05

                ax3.set_position(pie)
                                


                # plt.plot(x1[3:2160-3], array, 'b--')
                # plt.plot(x1, array2, 'r-')
                # plt.plot(x1[3:2160-3], agreement_score)
                # figname=f'{path_figs}{folder}/{item}_agreement.jpg'
                # plt.savefig(figname, bbox_inches='tight')
                plt.show()


## Print Agreement

In [136]:
plt.rcParams['figure.figsize'] = [80, 10]
mplstyle.use('fast')


folderlist, use_type, skipped_type, Excluded_paper, Excluded_agreement, KA_list_paper, KA_pre_cannula = get_figure_params( mouse_sort_csvs, use_expert=1 )

KA_list=KA_list_paper+KA_pre_cannula
Excluded=Excluded_agreement
print(Excluded)

# Controls: 644,645,646,647,
slope_sum=np.zeros((4,1))
avg_slope=np.zeros((4,1))

Controls=False
plot=True
global_count=0
maxlen=2160
for validation_only in [True, False]:

    if validation_only==True:
        print('Validation Dataset Agreement')
    else:
        print('Whole Dataset Agreement')
    for Controls in [False, True]:
        if Controls==False:
            print('Kainic Acid Treatment Agreement')
        else:
            print('Saline Control Agreement')

        dh_no, dh1, dh_maxagree, dh_minagree, dh_stddev, dh_conf = init_displays()
        global_agreement, global_confidence, max_agree, min_agree, n_animals = init_vars()
        agreement_list=[]
        std_dev=0
        global_count=0
        for folder in folderlist[::1]:
        
            
            Control=0
            KA_flag=False
            skip=0
            bad_flag=0
            Sz_occur=0
            mouse_folder = f'{mouse_sort_csvs}{folder}/'
            filelist=sorted(os.listdir(mouse_folder))
        
            count_state = np.zeros((len(filelist),6))
            count_state_total = np.zeros((len(filelist),5))
            cumu_sum = np.zeros((len(filelist),5))
        
            if os.path.isdir(f'{path_figs}{folder}')==False:
                os.mkdir(f'{path_figs}{folder}')
        
            filelist = [f for f in filelist if use_type in f]     
            for KA in KA_list:
                if f'{KA}' in folder:
                    KA_flag=True
        
            for Excl in Excluded:
                if f'{Excl}' in folder:
                    skip=1
    
            if validation_only==True:
                if folder not in testing_names:
                    skip=1
                
            if KA_flag!=Controls and skip==0:
                print(folder)
                sz=0
                total = len(filelist)
                print(total)
                arr = np.zeros((total,2))
                count=0
                
        
                if len(filelist)>0:
                    file_accuracy=np.ndarray((len(filelist),2), dtype='object')
                    for item in filelist[::1]:
                        if 'un' not in item:
        

                            
                            idx=filelist.index(item)
                            file_accuracy[idx,0]=item
                            
                            expert_array, classifier_array, conf = load_expert_and_classifier(mouse_folder, item)
            
                            agreement = array == array2[3:-3]
    
                            agree_pct=len(agreement[agreement==True])/len(agreement)
                        
                            file_accuracy[idx,1]=agree_pct
            
                            # np.savetxt(f'{basepath}/Results_GPU/{folder}_agreement_by_file.txt',file_accuracy,fmt='%s')
                            ind_name = item.find('NPM')
                            name = item[ind_name:ind_name+6]
                            date = item[ind_name+7:ind_name+13]
                            time = item[ind_name+14:ind_name+20]
                            
                            confidence_gradient=np.zeros(1)
                            agreement, min_agree, max_agree, global_agreement, agreement_list, std_dev, global_confidence, count, global_count = update_vars(expert_array, classifier_array, confidence_gradient, min_agree, max_agree, global_agreement, agreement_list, std_dev, global_confidence, count, global_count)
                            dh_no, dh1, dh_maxagree, dh_minagree, dh_stddev, dh_conf = update_displays(min_agree, max_agree, global_agreement, std_dev, global_confidence, count, global_count)
            
            
                
    


['NPM564', 'NPM565', 'NPM566', 'NPM567', 'NPM569', 'NPM570', 'NPM571', 'NPM572', 'NPM573', 'NPM574', 'NPM575', 'NPM576', 'NPM577', 'NPM578', 'NPM579', 'NPM580', 'NPM589', 'NPM590', 'NPM591', 'NPM592', 'NPM593', 'NPM594', 'NPM595', 'NPM596', 'NPM604', 'NPM605', 'NPM606', 'NPM607', 'NPM608', 'NPM609', 'NPM610', 'NPM611', 'NPM612', 'NPM613', 'NPM614', 'NPM615', 'NPM616', 'NPM617', 'NPM618', 'NPM619', 'NPM620', 'NPM621', 'NPM622', 'NPM623', 'NPM624', 'NPM625', 'NPM626', 'NPM627', 'NPM628', 'NPM629', 'NPM630', 'NPM631', 'NPM632', 'NPM633', 'NPM634', 'NPM635', 'NPM636', 'NPM637', 'NPM638', 'NPM639', 'NPM640', 'NPM641', 'NPM642', 'NPM644', 'NPM645', 'NPM646', 'NPM647', 'NPM648', 'NPM649', 'NPM650', 'NPM651', 'NPM652', 'NPM653', 'NPM654', 'NPM655', 'NPM656', 'NPM657', 'NPM658', 'NPM659', 'NPM661', 'NPM662', 'NPM663', 'NPM664', 'NPM665', 'NPM666', 'NPM667', 'NPM668']
[565, 568, 573, 578, 591, 594, 616, 618, 619, 623, 627, 630, 631, 662, 665, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564

'file: '

'agreement: '

'max agreement: '

'min agreement: '

NameError: name 'std_dev' is not defined

In [ ]:
folderlist, use_type, skipped_type, Excluded_paper, Excluded_agreement, KA_list_paper, KA_pre_cannula = get_figure_params( mouse_sort_csvs, use_expert=1 )

KA_list=KA_list_paper+KA_pre_cannula
Excluded=Excluded_agreement
print(Excluded)

slope_sum=np.zeros((4,1))
avg_slope=np.zeros((4,1))

Controls=False
plot=True
global_count=0
maxlen=2160
for validation_only in [True, False]:
    if validation_only==True:
        print('Validation Dataset Agreement')
    else:
        print('Whole Dataset Agreement')
    for Controls in [False, True]:
        if Controls==False:
            print('Kainic Acid Treatment Agreement')
        else:
            print('Saline Control Agreement')

        dh_no, dh1, dh_maxagree, dh_minagree, dh_stddev, dh_conf = init_displays()
        global_agreement, global_confidence, max_agree, min_agree, n_animals = init_vars()
        agreement_list=[]
        std_dev=0
        global_count=0
        for folder in folderlist[::1]:
            Control=0
            KA_flag=False
            skip=0
            bad_flag=0
            Sz_occur=0
            mouse_folder = f'{mouse_sort_csvs}{folder}/'
            filelist=sorted(os.listdir(mouse_folder))
        
            count_state = np.zeros((len(filelist),6))
            count_state_total = np.zeros((len(filelist),5))
            cumu_sum = np.zeros((len(filelist),5))
        
            if os.path.isdir(f'{path_figs}{folder}')==False:
                os.mkdir(f'{path_figs}{folder}')
        
            filelist = [f for f in filelist if use_type in f]     
            for KA in KA_list:
                if f'{KA}' in folder:
                    KA_flag=True
        
            for Excl in Excluded:
                if f'{Excl}' in folder:
                    skip=1
    
            if validation_only==True:
                if folder not in testing_names:
                    skip=1
                
            if KA_flag!=Controls and skip==0: